# Library import

In [1]:
import numpy as np
import pandas as pd 
import quandl as qu 
import QuantLib as ql
import datetime
import math 
import matplotlib.pyplot as plt
from scipy import interpolate
from tkinter import *
from pandastable import Table

# Time till maturity

In [4]:
def time_till_maturity(valuation_date,maturity):
    
    valuation_date = datetime.datetime.strptime(valuation_date.get(),"%m/%d/%Y")
    maturity = datetime.datetime.strptime(maturity.get(),"%m/%d/%Y")
    year= maturity.year - valuation_date.year

    moved_maturity = datetime.datetime(year=valuation_date.year,month=maturity.month,day=maturity.day)
    
    delta=(moved_maturity - valuation_date)
    year= year+ delta.days/day_count_convention.get()

    return year

# Zero rate curve

In [10]:
# For zero rate curve make a free quandl account and put authentication token where "*******" string is present 
def zero_rate_curve(valuation_date):
    
    valuation_date = datetime.datetime.strptime(valuation_date.get(),"%m/%d/%Y")
    tt_date = valuation_date+datetime.timedelta(days=1)
    
    zero_rate=qu.get("FED/SVENY", authtoken="********",start_date=valuation_date, end_date=tt_date)
    zc = zero_rate.iloc[0,:]
    zc=pd.DataFrame.from_dict({'rate':zc.values,"index":[int(i) for i in range(1,31)]})
    zc = zc.set_index('index')
    
    overnight_rate = qu.get("FED/RIFSPPNA2P2D01_N_B", authtoken="*********",start_date=valuation_date, end_date=tt_date)
    overnight_rate.rename(columns={'Value':'rate'},inplace=True)
    overnight_rate= overnight_rate.iloc[0,:]
    overnight_rate.name = 0
    zc = zc.append(overnight_rate, ignore_index=False)
    zc=zc.sort_index()
    return zc

In [6]:
def zero_curve_interpolation(x,y,kind):
    zc_interpolation = interpolate.interp1d(x,y,kind)
    return zc_interpolation

# Cash flow

In [7]:
def cash_flow(coupon_rate,face_value,coupon_frequency,year):
    
    coupon_rate = coupon_rate.get()
    coupon_frequency = coupon_frequency.get()
    first_coupon_date_fraction = year - math.floor(year)
    if first_coupon_date_fraction ==0:
        if coupon_frequency ==0:
            first_coupon_date_fraction =1
        elif coupon_frequency ==1:
            first_coupon_date_fraction =1
        elif coupon_frequency ==2:
            first_coupon_date_fraction =0.5
        elif coupon_frequency ==4:
            first_coupon_date_fraction =0.25
        elif coupon_frequency ==12:
            first_coupon_date_fraction =1/12
    
    
    
    if coupon_frequency ==0:
        time_fraction = 1
        cash_flow_cf = [ ((coupon_rate*face_value)*coupon_frequency) for i in range(math.ceil(year))]
        cash_flow_cf[-1]= cash_flow_cf[-1]+100
        cash_flow_time = [ np.round(first_coupon_date_fraction + i * time_fraction,2) for i in range(math.ceil(year)) ]

    else:
        time_fraction = 1/coupon_frequency
        cash_flow_cf = [ ((coupon_rate*face_value)/coupon_frequency) for i in range(math.ceil(year*coupon_frequency))]
        cash_flow_cf[-1]= cash_flow_cf[-1]+100
        cash_flow_time = [ np.round(first_coupon_date_fraction + i * time_fraction,2) for i in range(math.ceil(year*coupon_frequency)) ]
    
    return {"cash_flow":cash_flow_cf, 'cash_flow_time':cash_flow_time}

# Bond Pricer

In [8]:
def bond_pricer(coupon_rate,coupon_frequency,valuation_date,maturity):
    
    cf_dict=cash_flow(coupon_rate,face_value,coupon_frequency, year=time_till_maturity(valuation_date,maturity))
    zc = zero_rate_curve(valuation_date)
    zc_interpolation = zero_curve_interpolation(zc.index,zc['rate'],kind='linear')
    x_new = cf_dict['cash_flow_time']
    cf_dict['zero_rate']=zc_interpolation(x_new)/100
    df_cf = pd.DataFrame.from_dict(cf_dict)
    df_cf['discount_factor']=1/(1+df_cf['zero_rate'])**df_cf['cash_flow_time']
    df_cf['discounted_cash_flow']= df_cf['discount_factor']*df_cf['cash_flow']
    price=np.sum(df_cf['discounted_cash_flow'])
    
    return np.round(price,5),df_cf

# GUI

In [9]:
face_value =100
window = Tk()
window.geometry("1000x700") 
window.resizable(0, 0) # this prevents from resizing the window
window.title("Bond Pricer")

coupon_rate = DoubleVar()
# creating 2 text labels and input labels
Label(window, text = "Coupon rate").grid(row = 0,column=0) # this is placed in 0 0
# 'Entry' is used to display the input-field
Entry(window,textvariable=coupon_rate).grid(row = 0, column = 1) # this is placed in 0 1


coupon_frequency = IntVar()
coupon_frequency.set(2)
Label(window, text = "Coupon frequency").grid(row = 1,column=0) 
OptionMenu(window,coupon_frequency,0,1,2,4,12).grid(row=1,column=1)

trade_date = StringVar()
Label(window, text = "Trade date").grid(row = 2,column=0)
Entry(window,textvariable=trade_date).grid(row = 2, column = 1)



valuation_date= StringVar()

Label(window, text = "Valuation date").grid(row = 3,column=0) 
Entry(window,textvariable=valuation_date).grid(row = 3, column = 1)



maturity= StringVar()

Label(window, text = "Maturity date").grid(row = 4,column=0)
Entry(window,textvariable=maturity).grid(row = 4, column = 1)


day_count_convention = IntVar()
day_count_convention.set(365)
Label(window, text = "Day count convention").grid(row = 5,column=0) 
OptionMenu(window,day_count_convention,360,365).grid(row=5,column=1)


def getPrice():
    
    price,df_cf = bond_pricer(coupon_rate,coupon_frequency,valuation_date,maturity)
    Label(window,text="Price",font=('helvetica', 10, 'bold')).grid(row=7,column=0)
    Label(window, text= price,font=('helvetica', 10, 'bold')).grid(row=7,column=1)
    
    f = Frame()
    f.grid(row=8,column=1)
    pt = Table(f,dataframe=df_cf,showtoolbar=True, showstatusbar=True)
    pt.show()

my_button = Button(window, text = "Calculate",command= getPrice)
my_button.grid(row=6,column =1)

window.mainloop()

